### decision tree
#### is a supervised learning algorithm that is used for both classification and regression, called a tree bec its structured like one with:
- root node: the starting pt(the whole dataset)
- branches: decision rules based on features
- leaf nodes: the final outcomes (class labels or numeric predictions)

Think of it like a flowchart:

“If Temperature = Hot → check Humidity;
if Humidity = High → play = No; else → play = Yes.”

Just like KNN “looks” at neighbors to decide, a Decision Tree asks a series of yes/no questions to classify data.

### why use a decision tree?
Strength --- Explanation
Interpretable --- You can literally read the decision logic.
Non-linear --- Can handle non-linear relationships between input and output.
No need for scaling --- Works fine with raw data, no normalization required.
Works with categorical & numeric data --- Unlike KNN, which prefers numeric features.

but it also has weaknesses:
- can overfit (memorize training data)
- its sensitive to small data changes
- greedy algorithms, which may be sometimes suboptimal splits

### splitting the data
the tree keeps dividing/splitting the data at each step

each split tries to make the data in each subset as "pure" as possible, meaning that each branch should ideally contain one class

### entropy (used in ID3)
entropy measures disorder or impurity in a dataset, like if all the samples in a node belong to the same class the entropy of it will be equal to 0 (entropy=0); but if they are perfectly mixed then the entropy will be equal to 1 (entropy=1)
$$
Entropy(S) = -\sum_{i=1}^{n} p_i \log_2(p_i)
$$
where:
- \( S \) = dataset  
- \( n \) = number of classes  
- \( p_i \) = proportion of samples belonging to class \( i \)

**Example:**

If there are 9 positive and 5 negative samples:

$$
p(Yes) = \frac{9}{14}, \quad p(No) = \frac{5}{14}
$$

$$
Entropy(S) = -\left(\frac{9}{14}\log_2\frac{9}{14} + \frac{5}{14}\log_2\frac{5}{14}\right) = 0.94
$$

### information gain (ID3's heart)
information gain tells us how much entropy decreases when we split the data by some particular attribute

$$
IG(S, A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} \times Entropy(S_v)
$$

where:
- \( S \) = original dataset  
- \( A \) = attribute used for splitting  
- \( v \) = possible values of attribute \( A \)  
- \( S_v \) = subset of \( S \) for which \( A = v \)

higher information gain indicates better attribute to split on, hence the **ROOT NODE** is chosen by the **HIGHEST INFORMATION GAIN**


## STEPS TO IMPLEMENT DECISION TREE
1. Start with the root node containing the whole dataset.  
2. Use an Attribute Selection Measure (ASM) to choose the best attribute to split the node.  
3. Split the dataset into subsets for each possible value of the selected attribute.  
4. Create a decision node that contains the chosen attribute.  
5. Recursively repeat steps 1–4 for each child node until stopping criteria are met (all samples in a node have the same label or no attributes remain).

## ATTRIBUTE SELECTION MEASURE
Common ASMs:
- **Entropy / Information Gain** (used by ID3): chooses attributes that maximize reduction of entropy.  
- **Gini Index** (used by CART): uses impurity measure 1 − Σ(p_i²).  
- **Gain Ratio** (used by C4.5): normalizes Information Gain by SplitInfo to avoid bias toward attributes with many values.  
- Others: Chi-square, Reduction in Variance (for regression), etc.

## Implementing Decision Tree Classifier
- From-scratch approach: compute impurity (entropy/Gini), compute split quality (IG/Gini gain), select best attribute, recurse to build a nested structure (often returned as nested `dict`).  
- Library approach (scikit-learn): use `DecisionTreeClassifier`, choose `criterion` ('gini' or 'entropy'), tune hyperparameters (depth, leaf size, pruning).

## Training model using DT
- Fit the tree on training data using chosen ASM and hyperparameters.  
- If from-scratch, implement `fit()` by building the nested structure on the training dataset.

## Model Testing
- Use an unseen test set to compute predictions and evaluate performance metrics (accuracy, precision, recall, F1, confusion matrix).

## Model Training Accuracy & Model Testing Accuracy
- **Training accuracy**: how well the model fits the training data (can be high for overfitting).  
- **Testing accuracy**: measured on held-out data and is the main unbiased indicator of generalization.

## Decision Tree Classifier Parameters
- **CRITERION**: quality of split; `"gini"` (Gini impurity) or `"entropy"`/`"log_loss"` (information gain / Shannon entropy).  
- **SPLITTER**: strategy to choose split at each node: `"best"` (best split) or `"random"` (random best).  
- **MAX_DEPTH**: maximum depth of the tree; `None` lets it grow until leaves are pure or `min_samples_split` threshold.  
- **MIN_SAMPLES_SPLIT / MIN_SAMPLES_LEAF**: control minimal samples to split or to place in leaves (used to regularize).  
- **CCP_ALPHA**: complexity-cost pruning (post-pruning) parameter in sklearn — higher values produce smaller trees.

## Decision Tree using Entropy / Gini — with and without pruning
- *Without pruning*: build full tree until leaves are pure (or no attributes remain).  
- *With pruning*: restrict using `max_depth`, `min_samples_leaf`, or post-prune with `ccp_alpha` to reduce overfitting.

## How to Visualize a Decision Tree
- scikit-learn: `plot_tree`, `export_graphviz` + Graphviz, or `sklearn.tree.plot_tree`.  
- For from-scratch trees: write a small printer that prints the nested-dict in an indented form or convert to graph description.

### CODE
**1. High-level purpose — what these utilities do**

- **predict_single(tree, instance)** — given one example (a row), walk the nested-dict tree and return a predicted label.
- **predict_df(tree, df)** — call *predict_single* for every row in a DataFrame and return a numpy array of predictions.
- **evaluate_predictions(y_true, y_pred)** — compute and print model metrics (accuracy, precision/recall/F1, confusion matrix) using sklearn.metrics.
- **_global_default_label** — a fallback label used when the tree encounters missing or unseen attribute values.

These utilities let you take the tree that id3(...) produced and actually use it on new data and judge how well it performs.

**2. The tree data structure (very important)**

Your id3 function returns either:

A leaf: a label string like 'Yes' or 'No' (example: "Yes"), or

A dict with a single key: the attribute name, mapping to a dict of value -> subtree. 

Example:

{
  
  'Age': {
     
     'Young': {'Job_Status': {'Employed': 'No', 'Unemployed': 'Yes'}},
     
     'Senior': 'Yes',
     
     'Middle': 'Yes'
  
  }

}


So a node is either "leaf" or {'Attribute': {value1: subtree1, value2: subtree2, ...}}.

**predict_single** relies on that structure.

In [ ]:
def predict_single(tree_node, instance, default_label=_global_default_label):
    #1) leaf case: if the node is not a dict then its a label (terminal)
    if not isinstance(tree_node, dict):
        return tree_node
    
    #2) internal node: get the splitting attribute and its respective branches
    attr = next(iter(tree_node))
    branches = tree_node[attr]
    
    #3) read the instance's value for that attribute
    try:
        val = instance[attr]
    except Exception:
        return default_label
    
    #4) if the value was seen during training then recurse into that branch
    if val in branches:
        return predict_single(branches[val], instance, default_label)
    
    # 5) Unseen value handling: find majority label among reachable leaves,
    #    otherwise fall back to global default label.

Why each part?

- **Step 1**: stops recursion — leaf reached.
- **Step 2**: obtains which attribute this node uses to split. next(iter(...)) returns the first (and only) key.
- **Step 3**: tries to fetch the instance’s attribute value (works whether instance is a pd.Series row or dict).
- **Step 4**: normal path if the exact branch exists.
- **Step 5**: defensive strategy for unseen categories (common in real data): instead of crashing, we pick the majority label from all leaves under this node or fall back to the overall majority label.

**4 - The helper collect_leaves — what it does**

When we hit an unseen value, the code gathers all leaf labels under the current node to pick the majority. That function is just a recursive traversal:

In [ ]:
def collect_leaves(subnode):
    if not isinstance(subnode, dict):
        return [subnode]
    
    attr2 = next(iter(subnode))
    leaves = []
    
    for b in subnode[attr2].values():
        leaves.extend(collect_leaves(b))
    return leaves

This returns a list like ['Yes','Yes','No','Yes']. 

Counter(...).most_common(1)[0][0] picks the majority (e.g., 'Yes').

**5) predict_df and evaluate_predictions**

**predict_df** simply loops over rows and accumulates predictions (easy to understand; okay for small/mid datasets).

**evaluate_predictions** uses sklearn.metrics to print accuracy, a classification report (precision/recall/F1), and the confusion matrix. These are standard metrics you’ll need in your lab report.